<a href="https://colab.research.google.com/github/priscilahagunda/Machine-Learning-Labs/blob/main/TASK_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LassoCV
import io
import csv
from google.colab import files


Adding the CSV files

# Get the Dataset

In [ ]:
upload = files.upload()
data = pd.read_csv(io.BytesIO(upload['modified_data.csv']))

Saving modified_data.csv to modified_data (2).csv


Number of rows


In [ ]:
data.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,...,0,0,0,0,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,IR1,Lvl,AllPub,Inside,...,0,320,0,0,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,IR1,Lvl,AllPub,Corner,...,228,0,0,0,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,Reg,Lvl,AllPub,Inside,...,205,0,0,0,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,Reg,Lvl,AllPub,Corner,...,0,0,0,0,0,1,2008,WD,Normal,118000


In [ ]:
data.shape

(1460, 77)

In [ ]:
data.drop('Id',inplace=True,axis=1)

In [ ]:
data.head(5)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [ ]:
original_features=list(data.columns)
len(original_features)

76

In [ ]:
features_missing_values=list(data.columns[data.isna().any()])
len(features_missing_values)

15

In [ ]:
len(data)

1460

# Fill in Missing Features

In [ ]:
bfill,mean_fill=[],[]

In [ ]:
for feature in features_missing_values:
  print(feature,' ',data[feature].isna().sum()/len(data)*100,' ',data[feature].dtype)
  if data[feature].dtype=='float64':
    mean_fill.append(feature)
  else:
    bfill.append(feature)

LotFrontage   17.73972602739726   float64
MasVnrType   0.547945205479452   object
MasVnrArea   0.547945205479452   float64
BsmtQual   2.5342465753424657   object
BsmtCond   2.5342465753424657   object
BsmtExposure   2.6027397260273974   object
BsmtFinType1   2.5342465753424657   object
BsmtFinType2   2.6027397260273974   object
Electrical   0.0684931506849315   object
FireplaceQu   47.26027397260274   object
GarageType   5.5479452054794525   object
GarageYrBlt   5.5479452054794525   float64
GarageFinish   5.5479452054794525   object
GarageQual   5.5479452054794525   object
GarageCond   5.5479452054794525   object


In [ ]:
bfill

['MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond']

In [ ]:
mean_fill

['LotFrontage',
 'MasVnrArea',
 'GarageYrBlt',
 'LotFrontage',
 'MasVnrArea',
 'GarageYrBlt',
 'LotFrontage',
 'MasVnrArea',
 'GarageYrBlt']

In [ ]:
for feature in mean_fill:
  data[feature].fillna(data[feature].mean(),inplace=True)

In [ ]:
for feature in bfill:
  data[feature].fillna(method='bfill',inplace=True)

In [ ]:
data.columns[data.isna().any()]

Index([], dtype='object')

In [ ]:
data['GarageCond'].isna().sum()

0

In [ ]:
data['MasVnrArea'].isna().sum()

0

In [ ]:
data['GarageCond'].fillna(method='ffill',inplace=True)

# Encode the Dataset

In [ ]:
categorical_features=list(data.select_dtypes(include=['object']).columns)
categorical_features

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [ ]:
numerical=list(set(original_features)-set(categorical_features))
numerical

['EnclosedPorch',
 'GarageArea',
 'LotArea',
 'GarageYrBlt',
 'LowQualFinSF',
 'PoolArea',
 '1stFlrSF',
 '2ndFlrSF',
 'TotRmsAbvGrd',
 'MiscVal',
 'LotFrontage',
 'MoSold',
 'BsmtFinSF2',
 'MasVnrArea',
 'BsmtFullBath',
 'SalePrice',
 'YearBuilt',
 'WoodDeckSF',
 'YrSold',
 'OverallQual',
 'BsmtHalfBath',
 'Fireplaces',
 'YearRemodAdd',
 'GarageCars',
 'ScreenPorch',
 'TotalBsmtSF',
 'BsmtUnfSF',
 'BedroomAbvGr',
 'OpenPorchSF',
 'OverallCond',
 'GrLivArea',
 'MSSubClass',
 'HalfBath',
 'KitchenAbvGr',
 'FullBath',
 'BsmtFinSF1',
 '3SsnPorch']

In [ ]:
nominal=['MSZoning','LandContour','Neighborhood']
ordinal=list(set(categorical_features)-set(nominal))

In [ ]:
target=['SalePrice']

In [ ]:
df_nominal=pd.get_dummies(data[nominal])

In [ ]:
for feature in ordinal:
  data[feature]=data[feature].astype('category').cat.codes

df_ordinal=data[ordinal]

In [ ]:
data[numerical]

,EnclosedPorch,GarageArea,LotArea,GarageYrBlt,LowQualFinSF,PoolArea,1stFlrSF,2ndFlrSF,TotRmsAbvGrd,MiscVal,...,BedroomAbvGr,OpenPorchSF,OverallCond,GrLivArea,MSSubClass,HalfBath,KitchenAbvGr,FullBath,BsmtFinSF1,3SsnPorch
0,0,548,8450,2003.0,0,0,856,854,8,0,...,3,61,5,1710,60,1,1,2,706,0
1,0,460,9600,1976.0,0,0,1262,0,6,0,...,3,0,8,1262,20,0,1,2,978,0
2,0,608,11250,2001.0,0,0,920,866,6,0,...,3,42,5,1786,60,1,1,2,486,0
3,272,642,9550,1998.0,0,0,961,756,7,0,...,3,35,5,1717,70,0,1,1,216,0
4,0,836,14260,2000.0,0,0,1145,1053,9,0,...,4,84,5,2198,60,1,1,2,655,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0,460,7917,1999.0,0,0,953,694,7,0,...,3,40,5,1647,60,1,1,2,0,0
1456,0,500,13175,1978.0,0,0,2073,0,7,0,...,3,0,6,2073,20,0,1,2,790,0
1457,0,252,9042,1941.0,0,0,1188,1152,9,2500,...,4,60,9,2340,70,0,1,2,275,0
1458,112,240,9717,1950.0,0,0,1078,0,5,0,...,2,0,6,1078,20,0,1,1,49,0


In [ ]:
new_data=pd.concat([df_nominal,df_ordinal,data[numerical]],axis=1)

In [ ]:
new_data.shape

(1460, 107)

# Standardize

In [ ]:
new_data.drop('SalePrice',axis=1,inplace=True)

In [ ]:
X= new_data.to_numpy()

In [ ]:
X[0]

array([-0.08304548, -0.21585871, -0.10526316,  0.51813339, -0.41895507,
       -0.21235968, -0.18831089, -0.15899968,  0.33712564, -0.10854037,
       -0.03703704, -0.10526316, -0.20339487, -0.1398323 ,  2.95522137,
       -0.19025216, -0.27116307, -0.23917551, -0.16124951, -0.10854037,
       -0.1863522 , -0.42683279, -0.07875671, -0.22941573, -0.16998114,
       -0.23595776, -0.28963792, -0.13199092, -0.23106504, -0.20521398,
       -0.25018188, -0.13199092, -0.16347148, -0.08712888,  0.64046232,
       -0.12304604,  0.28974476,  1.02668924,  0.303692  , -0.23012211,
       -0.12579688, -0.22571613, -0.40936914, -0.03628912,  0.18797343,
       -0.49151573,  0.36420746,  0.28780036, -0.02618016,  0.06423821,
       -0.29055229,  0.22403433,  0.74309239,  0.30941909,  0.2085023 ,
        0.25597761,  0.83559903,  0.75162623, -0.41169079, -0.71775144,
       -0.77797579,  0.31386709, -0.40779461, -1.23804229,  0.60466978,
       -0.03174026,  1.11993319,  0.75073056, -0.88465767,  0.26

In [ ]:
X=StandardScaler().fit_transform(X)

In [ ]:
X[0]

array([-0.08304548, -0.21585871, -0.10526316,  0.51813339, -0.41895507,
       -0.21235968, -0.18831089, -0.15899968,  0.33712564, -0.10854037,
       -0.03703704, -0.10526316, -0.20339487, -0.1398323 ,  2.95522137,
       -0.19025216, -0.27116307, -0.23917551, -0.16124951, -0.10854037,
       -0.1863522 , -0.42683279, -0.07875671, -0.22941573, -0.16998114,
       -0.23595776, -0.28963792, -0.13199092, -0.23106504, -0.20521398,
       -0.25018188, -0.13199092, -0.16347148, -0.08712888,  0.64046232,
       -0.12304604,  0.28974476,  1.02668924,  0.303692  , -0.23012211,
       -0.12579688, -0.22571613, -0.40936914, -0.03628912,  0.18797343,
       -0.49151573,  0.36420746,  0.28780036, -0.02618016,  0.06423821,
       -0.29055229,  0.22403433,  0.74309239,  0.30941909,  0.2085023 ,
        0.25597761,  0.83559903,  0.75162623, -0.41169079, -0.71775144,
       -0.77797579,  0.31386709, -0.40779461, -1.23804229,  0.60466978,
       -0.03174026,  1.11993319,  0.75073056, -0.88465767,  0.26

In [ ]:
y=data[target]

In [ ]:
y=y.to_numpy()

In [ ]:
y.shape

(1460, 1)

In [ ]:
y

array([[208500],
       [181500],
       [223500],
       ...,
       [266500],
       [142125],
       [147500]])

In [ ]:
y=StandardScaler().fit_transform(y)

In [ ]:
y

array([[ 0.34727322],
       [ 0.00728832],
       [ 0.53615372],
       ...,
       [ 1.07761115],
       [-0.48852299],
       [-0.42084081]])

In [ ]:
X.shape

(1460, 106)

In [ ]:
X

array([[-0.08304548, -0.21585871, -0.10526316, ...,  0.78974052,
         0.57542484, -0.11633929],
       [-0.08304548, -0.21585871, -0.10526316, ...,  0.78974052,
         1.17199212, -0.11633929],
       [-0.08304548, -0.21585871, -0.10526316, ...,  0.78974052,
         0.09290718, -0.11633929],
       ...,
       [-0.08304548, -0.21585871, -0.10526316, ...,  0.78974052,
        -0.36987111, -0.11633929],
       [-0.08304548, -0.21585871, -0.10526316, ..., -1.02604084,
        -0.86554834, -0.11633929],
       [-0.08304548, -0.21585871, -0.10526316, ..., -1.02604084,
         0.84738933, -0.11633929]])

# Feature Extraction Using PCA

In [ ]:
pca=PCA(n_components=2)

In [ ]:
p_components=pca.fit_transform(X)

In [ ]:
p_components

array([[ 2.44330986, -1.27847651],
       [ 0.31646137,  2.63801909],
       [ 2.85722092, -0.81099783],
       ...,
       [ 1.26182885, -0.86094666],
       [-2.85096376,  3.29421511],
       [-0.77738419,  2.78945089]])

In [ ]:
pca.explained_variance_ratio_

array([0.10560233, 0.04472091])

# Feature selection and training the model

In [ ]:
reg=LassoCV()

In [ ]:
reg.fit(p_components,y)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:1571: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LassoCV()

In [ ]:
reg.coef_

array([0.25387739, 0.01886719])